In [1]:
import requests
import json
import numpy as np
import cvxpy as cp

import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

from items import sink_points, resource_production

import numpy as np
import pandas as pd
from collections import defaultdict


from items import sink_points, resource_production

resource_reserve = {"Electricity": 80000, "Singularity Cell": 20}
resource_maximum = {"Water": 100000}

recipes_url     = "https://satisfactory.wiki.gg/wiki/Template:DocsRecipes.json?action=raw"
items_url       = "https://satisfactory.wiki.gg/wiki/Template:DocsItems.json?action=raw"
buildings_url   = "https://satisfactory.wiki.gg/wiki/Template:DocsBuildings.json?action=raw"

recipes_data_   = json.loads(requests.get(recipes_url).text)
items_data_     = json.loads(requests.get(items_url).text)
buildings_data_ = json.loads(requests.get(buildings_url).text)

In [2]:
items_data, buildings_data = {}, {}

for name, alts in items_data_.items():
    item = [alt for alt in alts if alt["experimental"] == True][0]
    items_data[name] = item
    
for name, alts in buildings_data_.items():
    building = [alt for alt in alts if alt["experimental"] == True][0]
    buildings_data[name] = building

item_names          = {key: val['name'] for key, val in items_data.items() if 'name' in val}
building_names      = {key: val['name'] for key, val in buildings_data.items() if 'name' in val}
building_power_gen  = {key: val['powerGenerated'] for key, val in buildings_data.items() if 'powerGenerated' in val}
building_power_cons = {key: val['powerUsage'] for key, val in buildings_data.items() if 'powerUsage' in val}

In [3]:
recipes = []

for _, alts in recipes_data_.items():
    recipe = [alt for alt in alts if alt["experimental"] == True][0]
    name = recipe["name"]
    dur = recipe["duration"]
    inputs = recipe["ingredients"]
    outputs = recipe["products"]
    machine = recipe["producedIn"]
    if not machine:
        continue
    machine = machine[0]
    
    inputs_new, outputs_new = [], []
    for inp in inputs:
        inputs_new.append({
            "name": item_names[inp["item"]],
            "amount": 60.0*inp["amount"]/dur,
        })
    for out in outputs:
        outputs_new.append({
            "name": item_names[out["item"]],
            "amount": 60.0*out["amount"]/dur,
        })
    # min_power = recipe["minPower"]
    max_power = recipe["maxPower"]
    power_gen = building_power_gen[machine]
    power_cons = building_power_cons[machine]
    power_cons = max_power if max_power else power_cons
    overclockable = buildings_data[machine].get("overclockable", 0)
    double_output = buildings_data[machine].get("somersloopSlots", 0)
    if power_gen is not None and power_gen > 0:
        outputs_new.append({
            "name": "Electricity",
            "amount": power_gen,
        })
        overclockable = 0 
    if power_cons is not None and power_cons > 0:
        inputs_new.append({
            "name": "Electricity",
            "amount": power_cons,
        })
    recipes.append({
        "name": name,
        "inputs": inputs_new,
        "outputs": outputs_new,
        "machine": building_names[machine],
        "oc": overclockable and double_output,
        })

In [4]:
additional_recipes = [
    {
        "name": "Ficsonium Fuel Rod (burning)",
        "inputs": [
            {"name": "Ficsonium Fuel Rod", "amount": 1.0},
            {"name": "Water", "amount": 15.0}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 2500}
        ],
        "machine": "Nuclear Power Plant",
        "oc": False,
    },
    {
        "name": "Extract Water",
        "inputs": [
            {"name": "Electricity", "amount": 20}
        ],
        "outputs": [
            {"name": "Water", "amount": 120}
        ],
        "machine": "Water Extractor",
        "oc": False,
    },
    {
        "name": "Fuel (burning)",
        "inputs": [
            {"name": "Fuel", "amount": 20}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 250}
        ],
        "machine": "Fuel-Powered Generator",
        "oc": False,
    },
    {
        "name": "Liquid Biofuel (burning)",
        "inputs": [
            {"name": "Liquid Biofuel", "amount": 20}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 250}
        ],
        "machine": "Fuel-Powered Generator",
        "oc": False,
    },
    {
        "name": "Turbofuel (burning)",
        "inputs": [
            {"name": "Turbofuel", "amount": 7.5}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 250}
        ],
        "machine": "Fuel-Powered Generator",
        "oc": False,
    },
    {
        "name": "Rocket Fuel (burning)",
        "inputs": [
            {"name": "Rocket Fuel", "amount": 4.166667}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 250}
        ],
        "machine": "Fuel-Powered Generator",
        "oc": False,
    },
    {
        "name": "Ionized Fuel (burning)",
        "inputs": [
            {"name": "Ionized Fuel", "amount": 3}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 250}
        ],
        "machine": "Fuel-Powered Generator",
        "oc": False,
    },
    {
        "name": "Coal (burning)",
        "inputs": [
            {"name": "Coal", "amount": 15},
            {"name": "Water", "amount": 45}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 250}
        ],
        "machine": "Coal-Powered Generator",
        "oc": False,
    },
    {
        "name": "Compacted Coal (burning)",
        "inputs": [
            {"name": "Compacted Coal", "amount": 7.142857},
            {"name": "Water", "amount": 45}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 250}
        ],
        "machine": "Coal-Powered Generator",
        "oc": False,
    },
    {
        "name": "Petroleum Coke (burning)",
        "inputs": [
            {"name": "Petroleum Coke", "amount": 25},
            {"name": "Water", "amount": 45}
        ],
        "outputs": [
            {"name": "Electricity", "amount": 250}
        ],
        "machine": "Coal-Powered Generator",
        "oc": False,
    },
]
recipes.extend(additional_recipes)

In [5]:
# 构建所有物品全集，以及回收点和初始值

all_items = set()
all_items.add("Somersloop") 
for rec in recipes:
    for entry in rec['inputs'] + rec['outputs']:
        all_items.add(entry['name'])
item_list = sorted(all_items)
item_to_idx = {name: i for i, name in enumerate(item_list)}
n_items = len(item_list)

v = [0.0] * n_items
a = [0.0] * n_items
m = [0.0] * n_items

for name, val in sink_points:
    if name in item_to_idx:
        v[item_to_idx[name]] = val
    else:
        print(f"Warning: {name} not found in item list for sink points.")


for name, amt in resource_production:
    if name in item_to_idx:
        a[item_to_idx[name]] = amt
    else:
        print(f"Warning: {name} not found in item list for resource production.")
        
for resourece, amount in resource_reserve.items():
    if resourece in item_to_idx:
        a[item_to_idx[resourece]] = -amount
    else:
        print(f"Warning: {resourece} not found in item list for resource reserve.")
        
for resourece, amount in resource_maximum.items():
    if resourece in item_to_idx:
        m[item_to_idx[resourece]] = amount
    else:
        print(f"Warning: {resourece} not found in item list for resource maximum.")


In [6]:
#创建boost和normal的配方列表
normal_recipes = []
boost_recipes = []

for rec in recipes:
    normal_recipes.append({
        'name': rec['name'],
        'inputs': rec['inputs'],
        'outputs': rec['outputs']
    })
    if rec['oc'] > 0.5:
        # 增强版：2.5倍输入, 5倍输出, 13.431倍耗电, oc值消耗索莫晶体
        boost_outputs = [{'name': o['name'], 'amount': o['amount'] * 5} for o in rec['outputs']]
        boost_inputs = []
        for inp in rec['inputs']:
            if inp['name'] == 'Electricity':
                boost_inputs.append({'name': inp['name'], 'amount': inp['amount'] * 13.431})
            else:
                boost_inputs.append({'name': inp['name'], 'amount': inp['amount'] * 2.5})
        boost_inputs.append({'name': 'Somersloop', 'amount': rec['oc']})
        boost_recipes.append({
            'name': rec['name'] + ' (boost)',
            'inputs': boost_inputs,
            'outputs': boost_outputs,
        })

all_recipes = normal_recipes + boost_recipes
n_recipes = len(all_recipes)

In [7]:
# 构造 Δ 矩阵（n_items × n_recipes）
delta = [[0.0 for _ in range(n_recipes)] for _ in range(n_items)]

for j, rec in enumerate(all_recipes):
    for inp in rec['inputs']:
        idx = item_to_idx[inp['name']]
        delta[idx][j] -= inp['amount']
    for out in rec['outputs']:
        idx = item_to_idx[out['name']]
        delta[idx][j] += out['amount']

model = ConcreteModel()

model.normal_idx = Set(initialize=range(len(normal_recipes)))
model.boost_idx = Set(initialize=range(len(normal_recipes), n_recipes))

model.x = Var(range(n_recipes), domain=Any)

for i in model.normal_idx:
    model.x[i].domain = NonNegativeReals
for i in model.boost_idx:
    model.x[i].domain = NonNegativeIntegers



In [ ]:

# 目标函数：回收点数总量
def total_sink_points(model):
    total = 0.0
    for i in range(n_items):
        acc = a[i]  # 初始库存
        # 加上所有配方对该物品的产率影响
        for j in range(n_recipes):
            acc += delta[i][j] * model.x[j]
        total += v[i] * acc
    return total

model.obj = Objective(rule=total_sink_points, sense=maximize)


# 资源守恒约束：所有物品最终库存 ≥ 0 
def resource_constraint_rule(model, i):
    acc = a[i]
    for j in range(n_recipes):
        acc += delta[i][j] * model.x[j]
    return acc >= 0

model.resource_constraints = ConstraintList()
for i in range(n_items):
    model.resource_constraints.add(resource_constraint_rule(model, i))
    
    
model.zero_garbage = ConstraintList()
for i in range(n_items):
    if v[i] < 1e-8:  # 不可回收
        expr = a[i] + sum(delta[i][j] * model.x[j] for j in range(n_recipes))
        model.zero_garbage.add(expr == 0)

# 资源最大约束：所有资源的最终库存 ≤ 最大值
def resource_max_constraint_rule(model, i):
    acc = a[i]
    for j in range(n_recipes):
        acc += delta[i][j] * model.x[j]
    return acc <= m[i]
model.resource_max_constraints = ConstraintList()
for i in range(n_items):
    if m[i] > 0:  # 仅对有最大值限制的资源添加约束
        model.resource_max_constraints.add(resource_max_constraint_rule(model, i))
        
        

# 使用 Gurobi 求解 ，也可以自行替换
#solver = SolverFactory('gurobi')
solver = SolverFactory('glpk')

results = solver.solve(model, tee=True)

# 输出目标函数值
print(f"Total sink points = {model.obj():.2f}")



Read LP format model from file C:\Users\Haruk\AppData\Local\Temp\tmpmn6uqw0g.pyomo.lp
Reading time = 0.01 seconds
x1: 220 rows, 571 columns, 3932 nonzeros
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13600KF, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 220 rows, 571 columns and 3932 nonzeros
Model fingerprint: 0x1264313f
Variable types: 304 continuous, 267 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+04]
  Objective range  [5e+00, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 9e+04]
Presolve removed 101 rows and 79 columns
Presolve time: 0.01s
Presolved: 119 rows, 492 columns, 2273 nonzeros
Variable types: 258 continuous, 234 integer (0 binary)
Found heuristic solution: objective 2.894013e+08

Root relaxation: objective 4.030295e+08, 268 iterations, 0.00 seconds (0.00 wo

In [9]:
#获取配方执行次数
x_vals = [model.x[j]() for j in range(n_recipes)]  
recipe_exec_df = pd.DataFrame([
    {
        'Recipe': all_recipes[j]['name'],
        'Count' : x_vals[j],
        'Inputs': ', '.join(f"{inp['amount']:.2f} {inp['name']}" for inp in all_recipes[j]['inputs']),
        'Outputs': ', '.join(f"{out['amount']:.2f} {out['name']}" for out in all_recipes[j]['outputs'])
    }
    for j in range(n_recipes)
])

# 只显示用到的配方（执行次数大于 1e-4）
used_df = recipe_exec_df[recipe_exec_df['Count'] > 1e-4].sort_values(by="Count", ascending=False)
print(used_df.to_string(index=False))
used_df.to_csv("used_recipes.csv", index=False)

                      Recipe       Count                                                                                                                                                                Inputs                                                   Outputs
       Rocket Fuel (burning) 3992.277068                                                                                                                                                      4.17 Rocket Fuel                                        250.00 Electricity
                   Iron Wire 3482.792318                                                                                                                                    12.50 Iron Ingot, 4.00 Electricity                                                22.50 Wire
           Pure Copper Ingot 2200.882576                                                                                                                      15.00 Copper Ore, 10.00 Water, 30.00 Electricit

In [10]:
"""查询物品用法，按输入/输出分类并显示其他输入/输出项"""
"""Query the usage of a specific item in recipes, showing its role as input or output."""
 
def query_item_usage(item_name: str, all_recipes, x_vals, threshold=1e-6):
    used_as_input = []
    used_as_output = []

    for j, recipe in enumerate(all_recipes):
        count = x_vals[j]
        if count < threshold:
            continue

        inputs = recipe.get('inputs', [])
        outputs = recipe.get('outputs', [])
        input_names = [i['name'] for i in inputs]
        output_names = [o['name'] for o in outputs]

        # ➤ 作为输入
        for inp in inputs:
            if inp['name'] == item_name:
                total = inp['amount'] * count
                other_inputs = [i for i in inputs if i['name'] != item_name]
                used_as_input.append({
                    'name': recipe['name'],
                    'count': count,
                    'rate': inp['amount'],
                    'total': total,
                    'others_in': other_inputs,
                    'outputs': outputs
                })

        # ➤ 作为输出
        for out in outputs:
            if out['name'] == item_name:
                total = out['amount'] * count
                other_outputs = [o for o in outputs if o['name'] != item_name]
                used_as_output.append({
                    'name': recipe['name'],
                    'count': count,
                    'rate': out['amount'],
                    'total': total,
                    'others_out': other_outputs,
                    'inputs': inputs
                })

    print(f"\n 查询物品：{item_name}\n")

    if used_as_input:
        print("作为原料：")
        for r in sorted(used_as_input, key=lambda x: -x['total']):
            print(f"  - {r['name']:<35s} × {r['count']:.4f} → 消耗 {r['rate']:.2f} × {r['count']:.4f} = {r['total']:.4f}")
            if r['others_in']:
                print("    - 其他原料：")
                for i in r['others_in']:
                    print(f"      - {i['name']:<20s} {i['amount']:.2f} × {r['count']:.4f} = {i['amount'] * r['count']:.4f}")
            if r['outputs']:
                print("    - 所有产物：")
                for o in r['outputs']:
                    print(f"      - {o['name']:<20s} {o['amount']:.2f} × {r['count']:.4f} = {o['amount'] * r['count']:.4f}")
            print("")
    else:
        print("作为原料：无\n")

    if used_as_output:
        print("作为产物：")
        for r in sorted(used_as_output, key=lambda x: -x['total']):
            print(f"  - {r['name']:<35s} × {r['count']:.4f} → 产出 {r['rate']:.2f} × {r['count']:.4f} = {r['total']:.4f}")
            if r['others_out']:
                print("    - 其他产物：")
                for o in r['others_out']:
                    print(f"      - {o['name']:<20s} {o['amount']:.2f} × {r['count']:.4f} = {o['amount'] * r['count']:.4f}")
            if r['inputs']:
                print("    - 所有原料：")
                for i in r['inputs']:
                    print(f"      - {i['name']:<20s} {i['amount']:.2f} × {r['count']:.4f} = {i['amount'] * r['count']:.4f}")
            print("")
    else:
        print("作为产物：无\n")


In [11]:
#query_item_usage("Encased Plutonium Cell", all_recipes, x_vals)
query_item_usage("Iron Ore", all_recipes, x_vals)



 查询物品：Iron Ore

作为原料：
  - Pure Iron Ingot                     × 2158.1252 → 消耗 35.00 × 2158.1252 = 75534.3813
    - 其他原料：
      - Water                20.00 × 2158.1252 = 43162.5036
      - Electricity          30.00 × 2158.1252 = 64743.7554
    - 所有产物：
      - Iron Ingot           65.00 × 2158.1252 = 140278.1367

  - Sulfur (Iron)                       × 42.9369 → 消耗 300.00 × 42.9369 = 12881.0768
    - 其他原料：
      - Reanimated SAM       10.00 × 42.9369 = 429.3692
      - Electricity          400.00 × 42.9369 = 17174.7691
    - 所有产物：
      - Sulfur               120.00 × 42.9369 = 5152.4307

  - Basic Iron Ingot                    × 147.3817 → 消耗 25.00 × 147.3817 = 3684.5419
    - 其他原料：
      - Limestone            40.00 × 147.3817 = 5895.2670
      - Electricity          16.00 × 147.3817 = 2358.1068
    - 所有产物：
      - Iron Ingot           50.00 × 147.3817 = 7369.0838

作为产物：无



In [12]:
hide_idle = True  

item_df = pd.DataFrame({
    'Item': item_list,
    'Initial': a,
    'crafted': [sum(delta[i][j] * x_vals[j] for j in range(n_recipes) if delta[i][j] > 0) for i in range(n_items)],
    'Total': [a[i] + sum(delta[i][j] * x_vals[j] for j in range(n_recipes) if delta[i][j] > 0) for i in range(n_items)],
    'Used': [sum(delta[i][j] * x_vals[j] * -1.0 for j in range(n_recipes) if delta[i][j] < 0) for i in range(n_items)],
    'Unused': [a[i] + sum(delta[i][j] * x_vals[j] for j in range(n_recipes) if delta[i][j] > 0) - sum(delta[i][j] * x_vals[j] * -1.0 for j in range(n_recipes) if delta[i][j] < 0) for i in range(n_items)],
    'Sink Value': v,
})

if hide_idle:
        item_df = item_df[~((item_df['Initial'] < 1e-5) & (item_df['crafted'] < 1e-5) & (item_df['Used'] < 1e-5))]

item_df['Sink Points'] = (item_df['Total'] - item_df['Used']) * item_df['Sink Value']
total_sink_points = item_df['Sink Points'].sum() 
item_df = item_df.sort_values(by='Sink Points', ascending=False)
item_df = item_df.map(lambda x: f"{x:.4f}" if isinstance(x, float) else x)

print(f'Total Sink Points from all items: {total_sink_points:.2f}')
print(item_df.to_string(index=False))
item_df.to_csv("item_usage.csv", index=False)

Total Sink Points from all items: 402992783.79
                       Item     Initial      crafted        Total         Used   Unused   Sink Value    Sink Points
       Ballistic Warp Drive      0.0000      95.0000      95.0000       0.0000  95.0000 2895334.0000 275056730.0000
   Assembly Director System      0.0000     143.6369     143.6369       0.0000 143.6369  500176.0000  71843722.8914
        AI Expansion Server      0.0000      90.6250      90.6250       0.0000  90.6250  597652.0000  54162212.5000
         Plutonium Fuel Rod      0.0000      12.6000      12.6000       0.0000  12.6000  153184.0000   1930118.4000
              Supercomputer      0.0000     224.2619     224.2619     224.2619   0.0000   97352.0000         0.0000
                   Computer      0.0000    1441.3332    1441.3332    1441.3332   0.0000    8352.0000         0.0000
      Alclad Aluminum Sheet      0.0000    7560.2080    7560.2080    7560.2080   0.0000     266.0000         0.0000
Electromagnetic Control R

In [13]:
import plotly.graph_objects as go

not_included_items = {"Electricity", "Water", "Somersloop"}
included_items = sorted(set(item_list) - not_included_items)
item_to_sankey_idx = {name: i for i, name in enumerate(included_items)}

sources = []
targets = []
values = []
edge_labels = []
link_colors = []

for j, recipe in enumerate(all_recipes):
    try:
        count = model.x[j]()
    except:
        continue
    if count < 1e-4:
        continue

    # === 边颜色规则 ===
    if '(boost)' in recipe['name']:
        color = 'rgba(255, 0, 0, 0.4)'  # 红色
    elif 'Power' in recipe['name']:
        color = 'rgba(255, 165, 0, 0.4)'  # 橙色
    else:
        color = 'rgba(0, 128, 255, 0.3)'  # 蓝色

    for inp in recipe['inputs']:
        for out in recipe['outputs']:
            if inp['name'] in not_included_items or out['name'] in not_included_items:
                continue
            if inp['name'] not in item_to_sankey_idx or out['name'] not in item_to_sankey_idx:
                continue
            sources.append(item_to_sankey_idx[inp['name']])
            targets.append(item_to_sankey_idx[out['name']])
            values.append(out['amount'] * count)
            edge_labels.append(f"{recipe['name']} × {count:.2f}")
            link_colors.append(color)

fig = go.Figure(data=[go.Sankey(
    arrangement="snap",  
    node=dict(
        pad=18,
        thickness=12,
        line=dict(color="gray", width=0.6),
        label=included_items,
        hoverlabel=dict(bgcolor="white", font_size=13, font_family="Arial")
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        label=edge_labels,
        color=link_colors,
        hoverlabel=dict(bgcolor="white", font_size=14, font_family="Arial")
    )
)])

fig.update_layout(
    title=dict(
        text="物品流动图（红色为 Boost 配方）",
        font_size=20
    ),
    font=dict(size=13, color="black", family="Arial"),
    margin=dict(l=30, r=30, t=50, b=30),
    height=1500,
    plot_bgcolor="white",
    paper_bgcolor="white"
)

fig.show()

fig.write_html("sankey_diagram.html")
